# Cleaning Bank Marketing Campaign Data

Personal loans are a lucrative revenue stream for banks. We are going to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data we provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

# Start coding here...

In [2]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())


credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [3]:
client_raw = df.get(['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage'])
print(client_raw.info())
client_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   mortgage        41188 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB
None


,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


First we convert the data types in client_raw into the target data types. The data in 'credit_default' and 'mortgage' column should be booleans but now they are objects. 

In [4]:
# convert data in 'credit_default' column into booleans
client_raw['credit_default'] = client_raw.get('credit_default').apply(lambda x: True if x == 'yes' else False)
client_raw.head()
# covert data in 'mortgage' column into boolenas
client_raw['mortgage'] = client_raw.get('mortgage').apply(lambda x: True if x == 'yes' else False)
# modify data in 'job' column
client_raw['job'] = client_raw['job'].apply(lambda x: x.replace('.', '_'))
# modify data in 'education' column
client_raw['education'] = client_raw['education'].apply(lambda x: x.replace('.', '_'))
client_raw['education'] = client_raw['education'].apply(lambda x: x.replace('unknown', 'np_NaN'))
print(client_raw.head(10))
client_raw.to_csv('client.csv', index=False)

   client_id  age          job  ...            education credit_default  mortgage
0          0   56    housemaid  ...             basic_4y          False     False
1          1   57     services  ...          high_school          False     False
2          2   37     services  ...          high_school          False      True
3          3   40       admin_  ...             basic_6y          False     False
4          4   56     services  ...          high_school          False     False
5          5   45     services  ...             basic_9y          False     False
6          6   59       admin_  ...  professional_course          False     False
7          7   41  blue-collar  ...               np_NaN          False     False
8          8   24   technician  ...  professional_course          False      True
9          9   25     services  ...          high_school          False      True

[10 rows x 7 columns]


In [5]:
campaign_raw = df.get(['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'month', 'day'])
print(campaign_raw.info())
campaign_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  object
 5   campaign_outcome            41188 non-null  object
 6   month                       41188 non-null  object
 7   day                         41188 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.5+ MB
None


,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
0,0,1,261,0,nonexistent,no,may,13
1,1,1,149,0,nonexistent,no,may,19
2,2,1,226,0,nonexistent,no,may,23
3,3,1,151,0,nonexistent,no,may,27
4,4,1,307,0,nonexistent,no,may,3


In [6]:
# modify 'previous_outcome' column
campaign_raw['previous_outcome'] = campaign_raw['previous_outcome'].apply(lambda x: True if x == 'success' else False)
# modify 'campaign_outcome' column
campaign_raw['campaign_outcome'] = campaign_raw['campaign_outcome'].apply(lambda x: True if x == 'yes' else False)
# create 'last_contact_date' (2022-MM-DD) by combining 'day' and 'month'
    # convert data in 'month' and 'day' column into numbers with two digits as strings
campaign_raw['month'] = pd.to_datetime(campaign_raw['month'].str.title(), format='%b').dt.month.astype(str).str.zfill(2)
campaign_raw['day'] = campaign_raw['day'].astype(str).str.zfill(2)
    # concatenate into 2022-MM-DD
campaign_raw = campaign_raw.assign(last_contact_date = '2022-' + campaign_raw['month'] + '-' + campaign_raw['day'])
campaign_raw = campaign_raw.drop(columns = ['month', 'day'])
print(campaign_raw.head())
campaign_raw.to_csv('campaign.csv', index=False)

   client_id  number_contacts  ...  campaign_outcome  last_contact_date
0          0                1  ...             False         2022-05-13
1          1                1  ...             False         2022-05-19
2          2                1  ...             False         2022-05-23
3          3                1  ...             False         2022-05-27
4          4                1  ...             False         2022-05-03

[5 rows x 7 columns]


In [7]:
economics_raw = df.get(['client_id', 'cons_price_idx', 'euribor_three_months'])
print(economics_raw.info())
economics_raw.to_csv('economics.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB
None
